## Запуск мінімальної екпертної системи

В консолі CLIPS уже можна добавляти правила, факти та виконувати запуск. При використані *experta* потрібно створити об'єкт який буде симулювати в собі CLIPS 

In [1]:
from experta import KnowledgeEngine

class BasicExample(KnowledgeEngine):
    pass


engine = BasicExample()

В середині об'єкта створюються початкові факти, правила і функції, а запуск еспертної системи відбувається як операція над екземпляром об'єкта 

Запуск ЕС у середовищі CLIPS: 

```CLIPS
(reset)
(run)
```

Запуск ЕС у Python:

In [2]:
engine.reset()
engine.run()

## Створення сутностей, шаблонів та фактів 

В середовищі CLIPS існують *deftemplate* та *defclass* для опису шаблонів фактів та сутностей предметної області. При використані *experta* використовуються лише класи.

Cтворення шаблону animal та person в середовищі CLIPS:
```CLIPS
(deftemplate animal
             (slot name)
             (slot approx_weight)
             (slot approx_height)
             (slot tail)
             (slot character)
             (multislot related_animals))

(deftemplate person
             (slot name)
             (slot animal_kind)
             (slot age)
             (multislot friends)) 

```

Cтворення класу animal та person мовою Python:

In [3]:
from experta import Fact, Field

# Сутність повина наслідуватися від класу experta.Fact
class Animal(Fact):
    # атрибути класу повині бути типу experta.Filed та мати в описаний тип даних
    # тип даних може бути як стандартний для Python, так і бути описаним вручну
    # mandatory вказує на обов'язковість атрибуту в класі 
    name = Field(str, mandatory=True)
    approx_weight = Field(int, mandatory=True)
    approx_height = Field(int, mandatory=True)
    tail = Field(int, mandatory=True)
    character = Field(str, mandatory=True)
    related_animals = Field(list, mandatory=True)
    
    
class Person(Fact):
    name = Field(str, mandatory=False)
    animal_kind = Field(str, mandatory=True)
    age = Field(int, mandatory=True)
    friends = Field(list, mandatory=False)

Створимо декілька початкових фактів для ЕС. Розглянем створення через *deffacts* та *assert* 

CLIPS deffacts:
```CLIPS
(deffacts initfacts "Initial Facts"
             (dogs are our friends)
             (animal (name dog) (approx_weight 20) (approx_height 30) (tail 1)
              (related_animals cat wolf) (character strong)
             )
             (animal (name cat) (approx_weight 4) (approx_height 10) (tail 1)
              (character sweet) (related_animals dog wolf)
             )
             (animal (name wolf) (approx_weight 4) (approx_height 40) (character bad)
             (tail 1)
              (related_animals dog wolf)
             )
)
```

Python deffacts:

In [4]:
from experta import DefFacts

class Example1(KnowledgeEngine):
    @DefFacts()
    def init_pets(self):
        yield Animal(name='dog', approx_weight=20, approx_height=30, tail=1, character='strong', related_animals=['cat', 'wolf'])
        yield Animal(name='cat', approx_weight=4, approx_height=10, tail=1, character='sweet', related_animals=['wolf', 'dog'])      
        yield Animal(name='wolf', approx_weight=4, approx_height=40, tail=1, character='bad', related_animals=['wolf', 'cat'])

CLIPS assert:
```CLIPS
(assert (person (name Rick) (animal_kind dog) (age 3) (friends Tim Kitty)))
(assert (person (name Tim) (animal_kind dog) (age 4) (friends Rick)))
(assert (person (name Kitty) (animal_kind cat) (age 4) (friends Rick)))
(assert (person (animal_kind mollusc) (age 1) (friends)))
```

В Python точно так само зробити не можна, потрібно або створити функцію в середині об'єкта ЕС, або добавити факти в екзепляр ЕС.


Реалізація через функцію:

In [5]:
class Example2(KnowledgeEngine):
    def init_persons(self):
        self.declare(Person(name="Rick", animal_kind='dog', age=3, friends=['Tim', 'Kitty']))
        self.declare(Person(name="Tim", animal_kind='dog', age=4, friends=['Rick']))
        
engine = Example2()
engine.reset()
engine.init_persons()

Додавання фактів через екземпляр ЕС:

In [6]:
engine.declare(Person(name="Kitty", animal_kind='dog', age=4, friends=['Rick'])) 
engine.declare(Person(animal_kind='mollusc', age=1))                   

Person(animal_kind='mollusc', age=1)

Переглянем список фактів.
CLIPS:
```CLIPS
(facts)
```

Python:

In [7]:
engine.facts

FactList([(0, InitialFact()),
          (1,
           Person(name='Rick', animal_kind='dog', age=3, friends=frozenlist(['Tim', 'Kitty']))),
          (2,
           Person(name='Tim', animal_kind='dog', age=4, friends=frozenlist(['Rick',]))),
          (3,
           Person(name='Kitty', animal_kind='dog', age=4, friends=frozenlist(['Rick',]))),
          (4, Person(animal_kind='mollusc', age=1))])

Видалення фактів за номером в CLIPS:
```CLIPS
(retract 1 3)
```

В Python функція приймає лише одне значення тому прийдеться її викликати двічі для досягнення того ж ефекту:

In [8]:
engine.retract(1)
engine.retract(3) # якщо виконати команду ще раз, то це призведе до помилки бо факти за такими індексам відсутні 

Модифікуєм факт у середовищі CLIPS:
```CLIPS
(modify 2 (age 5))
```

Python:

In [9]:
engine.declare(engine.modify(engine.facts[2], age=5))
engine.facts

FactList([(0, InitialFact()),
          (4, Person(animal_kind='mollusc', age=1)),
          (5,
           Person(name='Tim', animal_kind='dog', age=5, friends=frozenlist(['Rick',])))])

Як бачимо, напряму редактувати факт не можна, його треба витягнути і заново вставити змінений, це призводить до зміщення індексації(факт 2 став 5), що може збити з толку в деяких випаках

Видалимо усі факти. CLIPS:
```CLIPS
(clear)
```
Python:

In [10]:
engine.facts.clear() # факти зберігаються в звичайному списку, і дана функція стандартна для списку Python
engine.facts

FactList()

## Створення правил

Я створюю клас ЕС, від якого буду наслідувати наступні приклади. В класі я добавлю початкові факти, це дозволить уникнути дублювання коду в наступних прикладах, а також дещо змінню класи сутностей

In [11]:
class Animal(Fact):
    name = Field(str, mandatory=True)
    approx_weight = Field(int, mandatory=False)
    approx_height = Field(int, mandatory=False)
    tail = Field(int, mandatory=True)
    water = Field(bool, mandatory=True)
    related_animals = Field(list, mandatory=True)

    
class Person(Fact):
    name = Field(str, mandatory=False)
    animal_kind = Field(str, mandatory=True)
    age = Field(int, mandatory=True)
    friends = Field(list, mandatory=False)
    

class ExampleRules(KnowledgeEngine):
    @DefFacts()
    def init_facts(self):
        yield (Animal(name='dog', tail=1, water=False, approx_weight=20, approx_height=30, related_animals=['wolf']))
        yield (Animal(name='wolf', tail=1, water=False,  approx_weight=4, approx_height=10, related_animals=['dog', 'koyote']))
        yield (Animal(name='shark', tail=1, water=True, approx_weight=4, approx_height=40, related_animals=['fish']))
        yield (Person(name="Rick", animal_kind='dog', age=3, friends=['Tim', 'Kitty']))
        yield (Person(name="Tim", animal_kind='cat', age=4, friends=['Rick']))
        yield (Person(name="Kitty", animal_kind='dog', age=4, friends=['Rick'])) 

##### Задайте правила для яких умовним елементом є зразок з певними символьними обмеженнями для знаходження впорядкованих або невпорядкованих фактів:

CLIPS:
```CLIPS
(defrule FindTailWaterAnimal
      (animal (tail 1) (life_environment water))
    =>
      (printout t crlf “Found tail water animal!” crlf)
    ) 
```
Python:

In [12]:
from experta import Rule

class Example(ExampleRules):
    # Правило задається за допомогою декоратора Rule
    @Rule(Animal(water=True))
    def findTailWaterAnimal(self):
        print('Found tail water animal!')
        
engine = Example()
engine.reset()
engine.run()

Found tail water animal!


##### Правило, зразку якого відповідають факти, що визначать тварин, які є родичами собаки:

CLIPS:
```CLIPS
(defrule FindDogRelative
      (animal (related_animals dog))
    =>
      (printout t crlf “Found dog relative!” crlf)
    ) 

```
У Python немає можливості зробити в точності як в CLIPS, для того щоб зробити перевірку ```'dog' in [...]``` потрібно виокремити зміну та працювати з нею, або в декораторі, або в тілі функції. Працювати із змінною в тілі функції простіше, так і поступим:

In [13]:
from experta import MATCH 
    
class Example(ExampleRules):
    # Match.name дозволяє передати name в функцію
    # Ім'я не обов'язково має співпадати з назвою атрибута сутності
    # Наприклад: name=MATCH.n передасть в функцію значення n
    # Якщо MATCH застосовується до зміни яку не треба передавати в середину функції, то її можна просто не приймати
    @Rule(Animal(related_animals=MATCH.related_animals))
    def findDogRelative(self, related_animals):
        if 'dog' in related_animals:
            print('Found dog relative!')
        
engine = Example()
engine.reset()
engine.run()

Found dog relative!


Команда *ppdefrule* у Python відсутня, як і збереження правил і фактів і вигляді файлівю Якщо потрібно подібний функціонал, то рекоменується розглянути питання серіалізації у мові Python, в часності документи **Yaml**

##### Задайте правила для яких умовним елементом є зразок в якому використовуються символьні обмеження та групові символи всіх видів для знаходження впорядкованих або невпорядкованих фактів.

    Правило, зразку якого відповідають факти, що визначають тварину з будь-яким ім’я у віці 3х років і яка товаришує з Kitty та, можливо, з іншими тваринами


CLIPS:
```CLIPS
(defrule FindKittyFriends
        (person (name ?) (age 3) (friends $? Kitty $?))
        =>
        (printout t crlf “Found Kitty friend!” crlf)
        ) 
```
Python:

In [14]:
class Example(ExampleRules):
    @Rule(Person(age=3, friends=MATCH.friends))
    def findKittyFriends(self, friends):
        if 'Kitty' in friends:
            print('Find Kitty Friends')
        
engine = Example()
engine.reset()
engine.run()

Find Kitty Friends


##### Задайте правила у яких використовуються зв’язуючі обмеження різних типів:

    Дане правило знайде всі пари тварин та їх друзів, які не є котами (кішками):


CLIPS:
```CLIPS
(defrule FindAnimals
        (person (name ?xname) (friends $? ?x $?))
        (person (name ?x) (animal_kind ~cat))
        =>
        (printout t crlf “Animal ”  ?xname “ and his friend (not cat) ” ?x crlf)
        )
```
Python:

In [15]:
from experta import L

# В Python не можна проводити тіж операції над строками, що у CLIPS, наприклад операція "не"
# Якщо нам потрібний такий функціонал, потрібно використати Literal - клас L з бібліотеки experta 

class Example(ExampleRules):
    @Rule(Person(animal_kind=~L('cat'), name=MATCH.name, friends=MATCH.friends))
    def findAnimals(self, name, friends):
        print(f'{name} and his friend (not cat) {", ".join(list(friends))}')
        
engine = Example()
engine.reset()
engine.run()

Kitty and his friend (not cat) Rick
Rick and his friend (not cat) Tim, Kitty


##### Задайте правила у яких використовуються предикатні обмеження:

    Дане правило знайде всіх тварин, що мають ім’я довжиною до 5 літер, та їх друзів, у яких ім’я починається літерою “T”:
CLIPS:
```CLIPS
(defrule PredicateFindAnimals
        (person (name ?xname&:(< (str-length ?xname) 5))
         (friends $? ?x&:(= 0 (str-compare (sub-string 1 1 (upcase ?x)) “T”)) $?))
        =>
        (printout t crlf “Animal ”  ?xname “ and his friend (‘T…’) ” ?x crlf)
        )
```
Python:

In [16]:
from experta import P, AS

# Предикат створюється за допомогою класу P який приймає булеву лямбда-функцию
# Оператор AS дозволяє передавати в середину методу цілу сутність, а не її атрибути як MATCH

class Example(ExampleRules):
    @Rule(
        AS.person << Person(
            name=P(lambda x: len(x) <  5),
            friends=P(lambda x: [True for name in x if name[0] == "T"])
        )
    )
    def PredicateFindAnimals(self, person):
        for fr in person.as_dict()["friends"]:
            print(f'{person.as_dict()["name"]} and his friend (T...) {fr}' )
        
engine = Example()
engine.reset()
engine.run()

Rick and his friend (T...) Tim
Rick and his friend (T...) Kitty


Задайте правила у яких використовуються обмеження, що повертають значення:

    Дане правило знайде всіх тварин у яких середня вага в 5 разів перевищує вік:
CLIPS:
```CLIPS
(defrule ReturnFindAnimals
        (person (name ?xname) (animal_kind ?ac) (age ?age))
        (animal (name ?ac) (approx_weight ?aw&=(* 5 ?age)))
        =>
        (printout t crlf “Animal ”  ?xname “, kind =  ” ?ac “, weight =  ” ?aw “, age =  ” ?age crlf)
        )
``` 
Python немає прямих аналогів таких обмежень, найкращим вибором буде використовувати клас TEST:

In [17]:
from experta import TEST

# TEST дозволяє створювати більш складні предикати
# MATCH передає зміні не тільки в функцію, але і у TEST

class Example(ExampleRules):
    @Rule(Person(animal_kind=MATCH.animal_kind, age=MATCH.age),
          Animal(name=MATCH.name, approx_weight=MATCH.approx_weight),
          TEST(lambda name, animal_kind: animal_kind == name),
          TEST(lambda age, approx_weight: age < approx_weight / 5)
         )
    def f(self, name, age, animal_kind, approx_weight):
        print(name, animal_kind, age, approx_weight)

engine = Example()
engine.reset()
engine.run()

dog dog 3 20


##### Задайте правила з використанням умовного елементу test.

	Це правило знаходить усі можливі пари тварин у яких перша тварина має більший зріст ніж друга і на екран виводяться назви цих тварин: 
CLIPS:
```CLIPS
(defrule TestRule
        (animal (name ?n1) (approx_height ?h1))
        (animal (name ?n2) (approx_height ?h2))
        (test (> ?h1 ?h2))
        =>
        (printout t crlf ?n1 " height > " ?n2 " height" crlf)
        )
```
Python:

In [18]:
class Example(ExampleRules):
    @Rule(
        Animal(approx_height=MATCH.approx_height1, name=MATCH.n1),
        Animal(approx_height=MATCH.approx_height2, name=MATCH.n2),
        TEST(lambda approx_height1, approx_height2: approx_height1 > approx_height2)
    )
    def testRule(self, n1, n2):
        print(n1, 'higest than', n2)
        
engine = Example()
engine.reset()
engine.run()

shark higest than wolf
shark higest than dog
dog higest than wolf


	Це правило знаходить усі можливі пари тварин, які мають хвіст та у яких перша тварина має не меншу вагу ніж друга і на екран виводяться назви цих тварин: 
CLIPS
```CLIPS
(defrule TestRule1
        (animal (name ?n1) (approx_weight ?h1) (tail 1))
        (animal (name ?n2) (approx_weight ?h2) (tail 1))
        (test (>= ?h1 ?h2))
        =>
        (printout t crlf ?n1 " weight >= " ?n2 " weight" crlf)
        )
```
Python

In [21]:
class Example(ExampleRules):
    @Rule(
        Animal(approx_weight=MATCH.approx1, name=MATCH.n1, tail=1),
        Animal(approx_weight=MATCH.approx2, name=MATCH.n2, tail=1),
        TEST(lambda approx1, approx2: approx1 >= approx2)
    )
    def testRule1(self, n1, n2):
        print(n1, 'more heavily than', n2)
        
engine = Example()
engine.reset()
engine.run()

wolf more heavily than shark
shark more heavily than wolf
dog more heavily than shark
shark more heavily than shark
dog more heavily than wolf
wolf more heavily than wolf
dog more heavily than dog
